In [27]:
import sys
import torch

sys.path.append('../')

In [28]:
from src.tacogfn.tasks import pharmaco_frag

In [33]:
hps = {
    "log_dir": "./logs/debug_run_pharmaco_frag_pb",
    "split_file": '../dataset/split_by_name.pt',
    "pharmacophore_db_path": '../misc/pharmacophores_db.lmdb',
    'affinity_predictor_path': "../logs/debug_docking_score_prediction_beta/model_state_23.pt",
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "overwrite_existing_exp": True,
    "num_training_steps": 10_000,
    "num_workers": 1,
    "opt": {
        "lr_decay": 20000,
    },
    "algo": {"sampling_tau": 0.99, "offline_ratio": 0.0},
    "cond": {
        "temperature": {
            "sample_dist": "uniform",
            "dist_params": [0, 64.0],
        }
    },
}

In [34]:
trainer = pharmaco_frag.PharmacophoreTrainer(hps)

loaded 15016 ids for train
loaded 100 ids for test


Hyperparameters:

log_dir: ./logs/debug_run_pharmaco_frag_pb
device: cuda
split_file: ../dataset/split_by_name.pt
pharmacophore_db_path: ../misc/pharmacophores_db.lmdb
affinity_predictor_path: ../logs/debug_docking_score_prediction_beta/model_state_23.pt
seed: 0
validate_every: 1000
checkpoint_every: null
print_every: 100
start_at_step: 0
num_final_gen_steps: null
num_training_steps: 10000
num_workers: 1
hostname: DESKTOP-2DNJF9F
pickle_mp_messages: false
git_hash: 6ae029a
overwrite_existing_exp: true
algo:
  method: TB
  global_batch_size: 64
  max_len: 128
  max_nodes: 9
  max_edges: 128
  illegal_action_logreward: -75.0
  offline_ratio: 0.0
  valid_offline_ratio: 0.0
  train_random_action_prob: 0.01
  valid_random_action_prob: 0.0
  valid_sample_cond_info: true
  sampling_tau: 0.99
  tb:
    bootstrap_own_reward: false
    epsilon: null
    reward_loss_multiplier: 1.0
    variant: TB
    do_correct_idempotent: false
    do_paramet

In [35]:
split_file = torch.load('../dataset/split_by_name.pt')
tuple_to_pharmaco_id = lambda t: t[0].split("/")[-1].split("_rec")[0]

train_ids = [tuple_to_pharmaco_id(t) for t in split_file["train"]]
test_ids = [tuple_to_pharmaco_id(t) for t in split_file["test"]]

In [36]:
for i in test_ids:
    if i not in trainer.pharmaco_db.all_id:
        print(f"Missing {i} in pharmacophore db")

In [6]:
trainer.print_every = 1

In [1]:
# trainer.run()